Define a binary tree data structure.

In [2]:
from collections import deque

class BinaryTreeNode:
    def __init__(self, data, left=None, right=None):
        self.data = data
        self.left = left
        self.right = right
        
    def height(self):
        left = self.left.height() if self.left else 0
        right = self.right.height() if self.right else 0
        return 1 + max(left, right)
        
    # TODO: rewrite this method when not tired
    def __str__(self):
        NODE_WIDTH = 4
        NODE_SPACE = 1
        h = self.height()
        max_num_leaves = 2**(h-1)
        total_width = max_num_leaves * NODE_WIDTH + (max_num_leaves - 1) * NODE_SPACE
        
        # Total space needed to print tree
        s = [list(" " * total_width) for _ in range(h*2-1)]
        
        def build_string(t, li, ri, level):
            data_str = str(t.data)[:NODE_WIDTH]
            node_str = "#" * (NODE_WIDTH - len(data_str)) + data_str
            
            # Position of text
            mi = li + (ri - li) // 2
            s[level][mi-2:mi+2] = list(node_str)
            
            if t.left:
                turn_idx = li + (mi-1-li) // 2
                s[level][turn_idx:mi-2] = list("_" * (mi-2-turn_idx))
                s[level+1][turn_idx] = "|"
                build_string(t.left, li, mi-0, level+2)
            if t.right:
                turn_idx = mi+1 + (ri - mi - 1) // 2
                s[level][mi+2:turn_idx] = list("_" * (turn_idx-mi-1))
                s[level+1][turn_idx] = "|"
                build_string(t.right, mi+1, ri, level+2)
        
        build_string(self, 0, total_width, 0)
        
        return "\n".join(["".join(arr) for arr in s])
            
        
# Shorthand
T = BinaryTreeNode

t = T(1, left=T(2), right=T(3, left=T(4), right=T(5)))
assert t.height() == 3
t.left.left = T(1) # Add node to left subtree, does not change height
assert t.height() == 3
t.right.right.right = T(6) # Add node to right subtree, increases height
assert t.height() == 4

t.right.right.right.right = T(99)
t.right.right.right.left = T(77)
t.right.right.left = T(55)
t.right.right.left.right = T(66)
t.right.right.left.left = T(44)
print(t)

                   __________________###1___________________                    
                   |                                       |                   
         ________###2                            ________###3_________          
         |                                       |                   |         
       ###1                                    ###4             ___###5____     
                                                                |         |    
                                                             _##55__   _###6__   
                                                             |     |   |     | 
                                                            ##44 ##66 ##77 ##99


Define graph data structures.

4.1 Implement a function to check if a binary tree is balanced. For the purposes of
this question, a balanced tree is defined to be a tree such that the heights of the
two subtrees of any node never differ by more than one.

In [12]:
def is_balanced(t):
    # TODO: balance usually means that the subtrees have to balanced as well, and cant just look at height
    # add test for this too
    #return abs(height(t.left) - height(t.right)) <= 1
    return False

t1 = T(1)
assert is_balanced(t1)
t2 = T(1, left=T(2))
assert is_balanced(t2)
t3 = T(1, right=T(2))
assert is_balanced(t3)
t4 = T(1, left=T(2, left=T(3), right=T(4)), right=T(5, left=T(6), right=T(7)))
assert is_balanced(t4)
t5 = T(1, left=T(2, left=T(3)))
assert not is_balanced(t5)

4.2 Given a directed graph, design an algorithm to find out whether there is a route
between two nodes.

In [27]:
from abc import ABCMeta, abstractmethod
from collections import defaultdict

class Graph(metaclass=ABCMeta):    
    @abstractmethod
    def add_node(self, u):
        return
    
    @abstractmethod
    def add_edge(self, u, v):
        return
    
    @abstractmethod
    def neighbors(self, u):
        return
    
    @abstractmethod
    def are_neighbors(self, u, v):
        return
    
    
class AdjacencyListGraph:
    def __init__(self, edges):
        self.adjacency_lists = defaultdict(list)
        for e in edges:
            #print(*e)
            self.add_edge(*e)
    
    def add_node(self, u):
        if u in self.adjacency_lists:
            return
        else:
            self.adjacency_lists[u] = []
    
    def add_edge(self, u, v):
        if v not in self.adjacency_lists[u]:
            self.adjacency_lists[u].append(v)
    
    def neighbors(self, u):
        return self.adjacency_lists[u] if u in self.adjacency_lists else []
    
    def are_neighbors(self, u, v):
        return v in self.adjacency_lists[u] if u in self.adjacency_lists else False
    
Graph.register(AdjacencyListGraph)

__main__.AdjacencyListGraph

In [29]:
from collections import deque
def has_route(g, u, v):
    # BFS, (DFS would also work)
    q = deque()
    q.append(u)
    visited = {}
    visited[u] = True
    while q:
        uu = q.popleft()
        for vv in g.neighbors(uu):
            if vv not in visited:
                if vv == v:
                    return True
                visited[vv] = True
                q.append(vv)
    return False

g1 = AdjacencyListGraph([(1, 2), (2, 3), (3, 4), (4, 5), (5, 6)])
assert has_route(g1, 1, 6)

g2 = AdjacencyListGraph([(1, 2), (2, 3), (4, 5), (5, 6)])
assert not has_route(g2, 1, 6)

4.3 Given a sorted (increasing order) array with unique integer elements, write an
algorithm to create a binary search tree with minimal height.

In [77]:
# Keep pivoting at mid since A is sorted and thus both sides will have minimum depth.
def to_bst(A):
    def bst(low, high):
        if low > high:
            return None
        elif low == high:
            return T(A[high])
        mid = low + (high - low) // 2
        left = bst(low, mid-1)
        right = bst(mid+1, high)
        return T(A[mid], left=left, right=right)
    return bst(0, len(A)-1)

t = to_bst([1, 2, 3, 10, 13, 99])
print(t)

    ___###3____     
    |         |    
  ###1_    _##13__   
      |    |     | 
    ###2 ##10  ##99


4.4 Given a binary tree, design an algorithm which creates a linked list of all the
nodes at each depth (e.g., if you have a tree with depth D, you'll have D linked
lists).

In [29]:
from collections import deque, defaultdict, Counter
def layer_lists(t):
    layers = defaultdict(deque)
    def traverse(t, layer):
        if not t:
            return
        traverse(t.left, layer+1)
        traverse(t.right, layer+1)
        layers[layer].append(t.data)
    
    traverse(t, 1)
    return layers

t = T(5, left=T(3, left=T(2), right=T(4)), right=T(8, left=T(7), right=T(9, left=T(10))))
print(t)
layers = layer_lists(t)
check = lambda layer, arr: Counter(layers[layer]) == Counter(arr)
assert check(1, [5])
assert check(2, [3, 8])
assert check(3, [2, 4, 7, 9])
assert check(4, [10])

         ________###5_________          
         |                   |         
    ___###3____         ___###8____      
    |         |         |         |    
  ###2      ###4      ###7     _###9   
                               |       
                              ##10     


4.5 Implement a function to check if a binary tree is a binary search tree

In [7]:
import math

def is_bst(t):
    def check_bst(t, mn, mx):
        if not t:
            return True
        if t.data < mn or t.data > mx:
            return False
        return check_bst(t.left, mn, t.data) and check_bst(t.right, t.data, mx)    
    
    return check_bst(t, -math.inf, math.inf)
        

t = T(5, left=T(3, left=T(2), right=T(4)), right=T(8, left=T(7), right=T(9)))
assert is_bst(t)
almost_bst = T(5, left=T(3, left=T(2), right=T(4)), right=T(8, left=T(4), right=T(9)))
assert not is_bst(almost_bst)

4.6 Write an algorithm to find the *next* node (i.e., in-order successor) of a given node
in a binary search tree. You may assume that each node has a link to its parent

In [5]:
class BinaryTreeNodeWithParent(BinaryTreeNode):
    def __init__(self, data, left=None, right=None):
        super().__init__(data, left, right)
        self.parent = None
    
    def set_parents(self):
        def traverse(t, p):
            if not t:
                return
            t.parent = p
            traverse(t.left, t)
            traverse(t.right, t)
            
        traverse(self, None)

# Shorthand
TP = BinaryTreeNodeWithParent      


def tree_next(t):
    if not t:
        return None
    
    if t.right:
        # If there is a right subtree, the next element is the leftmost node
        # in that subtree
        u = t.right
        while u.left:
            u = u.left
        return u
    else:
        # Keep going up as long as t is the right child, then pick parent of that
        while t.parent and t.parent.right == t:
            t = t.parent
        
        return t.parent
        

t = TP(5, left=TP(3, left=TP(2), right=TP(4)), right=TP(8, left=TP(7, left=TP(6)), right=TP(10, left=TP(9))))
t.set_parents()
print(t)

t2 = t.left.left
t3 = t.left
t4 = t.left.right
t5 = t
t6 = t.right.left.left
t7 = t.right.left
t8 = t.right
t9 = t.right.right.left
t10 = t.right.right
assert tree_next(t2) == t3
assert tree_next(t3) == t4
assert tree_next(t4) == t5
assert tree_next(t5) == t6
assert tree_next(t6) == t7
assert tree_next(t7) == t8
assert tree_next(t8) == t9
assert tree_next(t9) == t10
assert tree_next(t10) == None
assert tree_next(None) == None

         ________###5_________          
         |                   |         
    ___###3____         ___###8____      
    |         |         |         |    
  ###2      ###4     _###7     _##10   
                     |         |       
                    ###6      ###9     


4.7 Design an algorithm and write code to find the first common ancestor of two
nodes in a binary tree. Avoid storing additional nodes in a data structure. NOTE:
This is not necessarily a binary search tree.

In [76]:
# Stores no additional data
def common_ancestor(t, u):
    if not t or not u:
        return None
    if t == u:
        return t
    
    # First find which layer t and u are in
    tt = t
    t_layer = 0
    while tt:
        t_layer +=1
        tt = tt.parent
    uu = u
    u_layer = 0
    while uu:
        u_layer += 1
        uu = uu.parent
    
    # Then climb the difference in layer with the lower node
    layer_diff = abs(t_layer - u_layer)
    low_node, high_node = (t, u) if t_layer >= u_layer else (u, t)
    for _ in range(layer_diff):
        low_node = low_node.parent
    
    # Then climb with both until they are the same
    while low_node != high_node:
        low_node = low_node.parent
        high_node = high_node.parent
    
    return high_node
        

t = TP(5, left=TP(3, left=TP(2), right=TP(4)), right=TP(8, left=TP(7, left=TP(6)), right=TP(10, left=TP(9))))
t.set_parents()
print(t)

t2 = t.left.left
t3 = t.left
t4 = t.left.right
t5 = t
t6 = t.right.left.left
t7 = t.right.left
t8 = t.right
t9 = t.right.right.left
t10 = t.right.right
assert common_ancestor(t2, t4) == common_ancestor(t4, t2) == t3
assert common_ancestor(t2, t7) == t5
assert common_ancestor(t2, t9) == t5
assert common_ancestor(t6, t9) == t8
assert common_ancestor(t6, t8) == t8

         ________###5_________          
         |                   |         
    ___###3____         ___###8____      
    |         |         |         |    
  ###2      ###4     _###7     _##10   
                     |         |       
                    ###6      ###9     


In [77]:
def common_ancestor_with_additional_datastructure(t, u):
    pass

In [78]:
def common_ancestor_without_parents(t, u):
    pass

4.8 You have two very large binary trees: Tl, with millions of nodes, and T2, with
hundreds of nodes. Create an algorithm to decide ifT2 is a subtree of Tl.
A tree T2 is a subtree of Tl if there exists a node n in Tl such that the subtree of
n is identical to T2. That is, if you cut off the tree at node n, the two trees would
be identical.

In [ ]:
# TODO

4.9 You are given a binary tree in which each node contains a value. Design an algorithm to print all paths which sum to a given value. The path does not need to
start or end at the root or a leaf.

In [94]:
# TODO: not really right, but something like this I think
def sum_paths(t, S):
    def path(t, s, p):
        if s == 0:
            print(p)
        if s < 0:
            return
        if not t:
            return
        path(t.left, s - t.data, p + ["left"]) # TODO: Better indication of path taken
        path(t.left, s,  [])
        path(t.right, s - t.data, p + ["right"])
        path(t.right, s, [])
    
    path(t, S, [])

t = T(1, left=T(1, left=T(1)), right=T(2, left=T(5)))
print(t)
sum_paths(t, 3)

    ___###1____     
    |         |    
 _###1     _###2   
 |         |       
###1      ###5     
['left', 'left', 'left']
['left', 'left', 'right']
['right', 'left']
[]
[]
['right', 'right']
